In [1]:
import pandas as pd
import numpy as np
import time

# 引入贝叶斯优化库
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import clone

# ==========================================
# 1. 数据加载与分割 (80:20)
# ==========================================
file_path = 'C:/Users/tinid/polymer/major revision/标准化数据_无独热_Log变换12.18.csv'

try:
    data = pd.read_csv(file_path)
    print("数据加载成功 (UTF-8)")
except UnicodeDecodeError:
    data = pd.read_csv(file_path, encoding='gbk')
    print("数据加载成功 (GBK)")

excluded_columns = ['log_Separation factor', 'polymer', 'DOI', 'Flux']
X = data.drop(columns=excluded_columns, errors='ignore')
y = data['log_Separation factor']

# 划分出 20% 的独立测试集 (Test Set)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"原始数据总量: {len(X)}")
print(f"用于调优和CV的训练集 (80%): {len(X_train_full)}")
print(f"独立测试集 (20%): {len(X_test)}")

# ==========================================
# 2. 构建预处理与模型流水线
# ==========================================
categorical_features = ['Permeation type']
numerical_features = [col for col in X.columns if col not in categorical_features]

# 定义预处理器
# 数值列：使用 'passthrough' (因已标准化)
# 分类列：OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
    ])

# 定义基学习器 (Base Estimator)
# AdaBoost 默认使用 DecisionTreeRegressor(max_depth=3)
# 我们将在贝叶斯优化中调整它的 max_depth
base_estimator = DecisionTreeRegressor(random_state=42)

# 定义 Pipeline: 预处理 -> AdaBoost
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', AdaBoostRegressor(estimator=base_estimator, random_state=42))
])

# ==========================================
# 3. 第一阶段：贝叶斯超参数优化
# ==========================================
print("\n" + "="*50)
print(">>> Stage 1: 开始 AdaBoost 贝叶斯优化 (BayesSearchCV)...")
print("="*50)

# 定义搜索空间
# 注意：estimator__max_depth 是指 AdaBoost 内部决策树的深度
search_spaces = {
    'regressor__n_estimators': Integer(50, 500),          # 迭代次数
    'regressor__learning_rate': Real(0.001, 1.0, prior='log-uniform'), # 学习率
    'regressor__loss': Categorical(['linear', 'square', 'exponential']), # 损失函数
    'regressor__estimator__max_depth': Integer(1, 10),    # 基学习器(树)的深度
    'regressor__estimator__min_samples_split': Integer(2, 10) # 防过拟合
}

# 初始化贝叶斯搜索
opt = BayesSearchCV(
    pipeline,
    search_spaces,
    n_iter=30,      # 迭代 30 次
    cv=5,           # 调优阶段内部 5 折
    n_jobs=-1,      # 并行计算 (AdaBoost fit本身串行，但CV可并行)
    random_state=42,
    scoring='neg_mean_absolute_error',
    verbose=0
)

start_time = time.time()
opt.fit(X_train_full, y_train_full)
end_time = time.time()

best_estimator = opt.best_estimator_ # 包含预处理的完整 Pipeline
best_params = opt.best_params_

print(f"贝叶斯优化耗时: {end_time - start_time:.2f} 秒")
print("\n>>> 最佳参数:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

# ==========================================
# 4. 第二阶段：十折交叉验证 (使用最佳参数)
# ==========================================
print("\n" + "="*50)
print(">>> Stage 2: 使用最佳参数进行十折交叉验证 (10-Fold CV)...")
print("="*50)

kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    # 4.1 切分数据
    X_train_fold = X_train_full.iloc[train_idx]
    y_train_fold = y_train_full.iloc[train_idx]
    X_val_fold = X_train_full.iloc[val_idx]
    y_val_fold = y_train_full.iloc[val_idx]
    
    # 4.2 克隆最佳模型 (确保每一折独立训练)
    model = clone(best_estimator)
    
    # 4.3 训练 (Pipeline 自动处理 OneHot)
    model.fit(X_train_fold, y_train_fold)
    
    # 4.4 预测
    y_train_pred = model.predict(X_train_fold)
    y_val_pred = model.predict(X_val_fold)
    
    # 4.5 记录指标
    metrics = {
        "Fold": fold + 1,
        "Train R2": r2_score(y_train_fold, y_train_pred),
        "Train RMSE": np.sqrt(mean_squared_error(y_train_fold, y_train_pred)),
        "Train MAE": mean_absolute_error(y_train_fold, y_train_pred),
        "Val R2": r2_score(y_val_fold, y_val_pred),
        "Val RMSE": np.sqrt(mean_squared_error(y_val_fold, y_val_pred)),
        "Val MAE": mean_absolute_error(y_val_fold, y_val_pred)
    }
    fold_results.append(metrics)
    print(f"Fold {fold+1}/10 完成 | Val R2: {metrics['Val R2']:.4f}")

# 输出详细表格
df_cv_results = pd.DataFrame(fold_results)
avg_row = df_cv_results.mean(numeric_only=True).to_frame().T
avg_row["Fold"] = "Average"
df_final_cv = pd.concat([df_cv_results, avg_row], ignore_index=True)

print("\n>>> 十折交叉验证详细结果 (Per-Fold Results):")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(df_final_cv.round(4))

# ==========================================
# 5. 第三阶段：独立测试集最终评估
# ==========================================
print("\n" + "="*50)
print(">>> Stage 3: 独立测试集 (Hold-out Test Set) 最终评估...")
print("="*50)

# 使用 80% 的全量数据重新训练最佳模型
best_estimator.fit(X_train_full, y_train_full)

# 在 20% 独立测试集上预测
y_test_pred = best_estimator.predict(X_test)

final_summary = {
    "Metric": ["MAE", "RMSE", "R2"],
    "CV Training (Avg)": [
        df_final_cv.iloc[-1]["Train MAE"],
        df_final_cv.iloc[-1]["Train RMSE"],
        df_final_cv.iloc[-1]["Train R2"]
    ],
    "CV Validation (Avg)": [
        df_final_cv.iloc[-1]["Val MAE"],
        df_final_cv.iloc[-1]["Val RMSE"],
        df_final_cv.iloc[-1]["Val R2"]
    ],
    "Test Set (Final)": [
        mean_absolute_error(y_test, y_test_pred),
        np.sqrt(mean_squared_error(y_test, y_test_pred)),
        r2_score(y_test, y_test_pred)
    ]
}

print("\n>>> 模型性能最终汇总 (AdaBoost + BayesOpt):")
print(pd.DataFrame(final_summary).round(4))

数据加载成功 (UTF-8)
原始数据总量: 816
用于调优和CV的训练集 (80%): 652
独立测试集 (20%): 164

>>> Stage 1: 开始 AdaBoost 贝叶斯优化 (BayesSearchCV)...
贝叶斯优化耗时: 69.83 秒

>>> 最佳参数:
  regressor__estimator__max_depth: 10
  regressor__estimator__min_samples_split: 2
  regressor__learning_rate: 1.0
  regressor__loss: exponential
  regressor__n_estimators: 500

>>> Stage 2: 使用最佳参数进行十折交叉验证 (10-Fold CV)...
Fold 1/10 完成 | Val R2: 0.7607
Fold 2/10 完成 | Val R2: 0.7173
Fold 3/10 完成 | Val R2: 0.7382
Fold 4/10 完成 | Val R2: 0.9030
Fold 5/10 完成 | Val R2: 0.8782
Fold 6/10 完成 | Val R2: 0.8994
Fold 7/10 完成 | Val R2: 0.8066
Fold 8/10 完成 | Val R2: 0.7713
Fold 9/10 完成 | Val R2: 0.8946
Fold 10/10 完成 | Val R2: 0.7552

>>> 十折交叉验证详细结果 (Per-Fold Results):
       Fold  Train R2  Train RMSE  Train MAE  Val R2  Val RMSE  Val MAE
0         1    0.9979      0.0333     0.0204  0.7607    0.3740   0.2084
1         2    0.9976      0.0371     0.0233  0.7173    0.2903   0.1575
2         3    0.9977      0.0354     0.0220  0.7382    0.3658   0.1908
3      

In [2]:
import pandas as pd
import numpy as np
import time

# 引入贝叶斯优化库
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import clone

# ==========================================
# 1. 数据加载与分割 (80:20)
# ==========================================
file_path = 'C:/Users/tinid/polymer/major revision/通量标准化数据_无独热_Log变换12.18.csv'
data = pd.read_csv(file_path)

# Define excluded columns (unchanged)
excluded_columns = ['log_Flux', 'polymer', 'DOI', 'Separation factor']
X = data.drop(columns=excluded_columns, errors='ignore')
y = data['log_Flux']

# 划分出 20% 的独立测试集 (Test Set)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"原始数据总量: {len(X)}")
print(f"用于调优和CV的训练集 (80%): {len(X_train_full)}")
print(f"独立测试集 (20%): {len(X_test)}")

# ==========================================
# 2. 构建预处理与模型流水线
# ==========================================
categorical_features = ['Permeation type']
numerical_features = [col for col in X.columns if col not in categorical_features]

# 定义预处理器
# 数值列：使用 'passthrough' (因已标准化)
# 分类列：OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
    ])

# 定义基学习器 (Base Estimator)
# AdaBoost 默认使用 DecisionTreeRegressor(max_depth=3)
# 我们将在贝叶斯优化中调整它的 max_depth
base_estimator = DecisionTreeRegressor(random_state=42)

# 定义 Pipeline: 预处理 -> AdaBoost
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', AdaBoostRegressor(estimator=base_estimator, random_state=42))
])

# ==========================================
# 3. 第一阶段：贝叶斯超参数优化
# ==========================================
print("\n" + "="*50)
print(">>> Stage 1: 开始 AdaBoost 贝叶斯优化 (BayesSearchCV)...")
print("="*50)

# 定义搜索空间
# 注意：estimator__max_depth 是指 AdaBoost 内部决策树的深度
search_spaces = {
    'regressor__n_estimators': Integer(50, 500),          # 迭代次数
    'regressor__learning_rate': Real(0.001, 1.0, prior='log-uniform'), # 学习率
    'regressor__loss': Categorical(['linear', 'square', 'exponential']), # 损失函数
    'regressor__estimator__max_depth': Integer(1, 10),    # 基学习器(树)的深度
    'regressor__estimator__min_samples_split': Integer(2, 10) # 防过拟合
}

# 初始化贝叶斯搜索
opt = BayesSearchCV(
    pipeline,
    search_spaces,
    n_iter=30,      # 迭代 30 次
    cv=5,           # 调优阶段内部 5 折
    n_jobs=-1,      # 并行计算 (AdaBoost fit本身串行，但CV可并行)
    random_state=42,
    scoring='neg_mean_absolute_error',
    verbose=0
)

start_time = time.time()
opt.fit(X_train_full, y_train_full)
end_time = time.time()

best_estimator = opt.best_estimator_ # 包含预处理的完整 Pipeline
best_params = opt.best_params_

print(f"贝叶斯优化耗时: {end_time - start_time:.2f} 秒")
print("\n>>> 最佳参数:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

# ==========================================
# 4. 第二阶段：十折交叉验证 (使用最佳参数)
# ==========================================
print("\n" + "="*50)
print(">>> Stage 2: 使用最佳参数进行十折交叉验证 (10-Fold CV)...")
print("="*50)

kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    # 4.1 切分数据
    X_train_fold = X_train_full.iloc[train_idx]
    y_train_fold = y_train_full.iloc[train_idx]
    X_val_fold = X_train_full.iloc[val_idx]
    y_val_fold = y_train_full.iloc[val_idx]
    
    # 4.2 克隆最佳模型 (确保每一折独立训练)
    model = clone(best_estimator)
    
    # 4.3 训练 (Pipeline 自动处理 OneHot)
    model.fit(X_train_fold, y_train_fold)
    
    # 4.4 预测
    y_train_pred = model.predict(X_train_fold)
    y_val_pred = model.predict(X_val_fold)
    
    # 4.5 记录指标
    metrics = {
        "Fold": fold + 1,
        "Train R2": r2_score(y_train_fold, y_train_pred),
        "Train RMSE": np.sqrt(mean_squared_error(y_train_fold, y_train_pred)),
        "Train MAE": mean_absolute_error(y_train_fold, y_train_pred),
        "Val R2": r2_score(y_val_fold, y_val_pred),
        "Val RMSE": np.sqrt(mean_squared_error(y_val_fold, y_val_pred)),
        "Val MAE": mean_absolute_error(y_val_fold, y_val_pred)
    }
    fold_results.append(metrics)
    print(f"Fold {fold+1}/10 完成 | Val R2: {metrics['Val R2']:.4f}")

# 输出详细表格
df_cv_results = pd.DataFrame(fold_results)
avg_row = df_cv_results.mean(numeric_only=True).to_frame().T
avg_row["Fold"] = "Average"
df_final_cv = pd.concat([df_cv_results, avg_row], ignore_index=True)

print("\n>>> 十折交叉验证详细结果 (Per-Fold Results):")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(df_final_cv.round(4))

# ==========================================
# 5. 第三阶段：独立测试集最终评估
# ==========================================
print("\n" + "="*50)
print(">>> Stage 3: 独立测试集 (Hold-out Test Set) 最终评估...")
print("="*50)

# 使用 80% 的全量数据重新训练最佳模型
best_estimator.fit(X_train_full, y_train_full)

# 在 20% 独立测试集上预测
y_test_pred = best_estimator.predict(X_test)

final_summary = {
    "Metric": ["MAE", "RMSE", "R2"],
    "CV Training (Avg)": [
        df_final_cv.iloc[-1]["Train MAE"],
        df_final_cv.iloc[-1]["Train RMSE"],
        df_final_cv.iloc[-1]["Train R2"]
    ],
    "CV Validation (Avg)": [
        df_final_cv.iloc[-1]["Val MAE"],
        df_final_cv.iloc[-1]["Val RMSE"],
        df_final_cv.iloc[-1]["Val R2"]
    ],
    "Test Set (Final)": [
        mean_absolute_error(y_test, y_test_pred),
        np.sqrt(mean_squared_error(y_test, y_test_pred)),
        r2_score(y_test, y_test_pred)
    ]
}

print("\n>>> 模型性能最终汇总 (AdaBoost + BayesOpt):")
print(pd.DataFrame(final_summary).round(4))

原始数据总量: 791
用于调优和CV的训练集 (80%): 632
独立测试集 (20%): 159

>>> Stage 1: 开始 AdaBoost 贝叶斯优化 (BayesSearchCV)...
贝叶斯优化耗时: 66.29 秒

>>> 最佳参数:
  regressor__estimator__max_depth: 10
  regressor__estimator__min_samples_split: 2
  regressor__learning_rate: 1.0
  regressor__loss: exponential
  regressor__n_estimators: 332

>>> Stage 2: 使用最佳参数进行十折交叉验证 (10-Fold CV)...
Fold 1/10 完成 | Val R2: 0.3909
Fold 2/10 完成 | Val R2: 0.8771
Fold 3/10 完成 | Val R2: 0.4785
Fold 4/10 完成 | Val R2: 0.6297
Fold 5/10 完成 | Val R2: 0.6821
Fold 6/10 完成 | Val R2: 0.8373
Fold 7/10 完成 | Val R2: 0.7180
Fold 8/10 完成 | Val R2: 0.7946
Fold 9/10 完成 | Val R2: 0.8444
Fold 10/10 完成 | Val R2: 0.5898

>>> 十折交叉验证详细结果 (Per-Fold Results):
       Fold  Train R2  Train RMSE  Train MAE  Val R2  Val RMSE  Val MAE
0         1    0.9935      0.0501     0.0302  0.3909    0.4646   0.2447
1         2    0.9926      0.0541     0.0320  0.8771    0.2021   0.1505
2         3    0.9913      0.0587     0.0357  0.4785    0.3916   0.2311
3         4    0.9899 